In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import talib

from sklearn.model_selection import train_test_split
from utils import get_model_inputs, seed_everything, load_stock_data
from models import RNNmodel

device = torch.device("cuda")

In [4]:
full_data = pd.read_csv('../input/asx100/ASX100.csv')

input_dim = 10
hidden_dim = 2000
num_layers = 1 
rnn_type = 'gru'

n_back = 20
n_forward = 5

val_split = 0.2
batch_size = 7000 
num_epochs = 100
learning_rate = 0.7

seed_everything(1)

train_data, test_data, scalers = load_stock_data(full_data, n_back, test_date_split="2019-01-01", TI=True)
all_stocks = train_data.keys()

input_data = {stock: get_model_inputs(stock, train_data, n_back, n_forward) for stock in all_stocks}

x = np.concatenate([input_data[stock][0] for stock in all_stocks], axis=0)
y = np.concatenate([input_data[stock][1] for stock in all_stocks], axis=0)

x_train, x_val, y_train, y_val = [torch.from_numpy(a) for a in train_test_split(x, y, test_size=val_split, random_state=42)]


train = torch.utils.data.TensorDataset(x_train, y_train)
train_loader = torch.utils.data.DataLoader(dataset=train, 
                                           batch_size=batch_size, 
                                           shuffle=False,
                                           num_workers = 0)

val = torch.utils.data.TensorDataset(x_val, y_val)
val_loader = torch.utils.data.DataLoader(dataset=val, 
                                           batch_size=batch_size, 
                                           shuffle=False,
                                           num_workers=0)

model = RNNmodel(input_dim, hidden_dim, num_layers, n_forward, rnn_type, device)


optimizer = torch.optim.SGD(model.rnn.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=10, threshold=0.00005)

In [ ]:
losses = []
losses_val = []
print_every = 1
for epoch in range(1, num_epochs+1):
    for i, (input_tensor, target_tensor) in enumerate(train_loader):
        input_tensor, target_tensor = input_tensor.to(device), target_tensor.to(device)
        
        loss = model.train_step(input_tensor, target_tensor, optimizer)
        
        loss_val = 0
        for j, (val_input_tensor, val_target_tensor) in enumerate(val_loader):
            val_input_tensor, val_target_tensor = val_input_tensor.to(device), val_target_tensor.to(device)
            loss_val += model.validate(val_input_tensor, val_target_tensor).item()
        loss_val /= len(val_loader)
        
        print(".", end='')
        losses.append(loss)    
        losses_val.append(loss_val)  
        scheduler.step(loss_val)
        
        if epoch % print_every == 0 and i == len(train_loader)-1:
            print('Epoch %d [%d/%d] loss: %.6f loss_val: %.6f lr: %.10f' % (epoch, i, len(train_loader)-1, loss, loss_val, optimizer.param_groups[0]['lr']))
#     if epoch % 10 == 0:
#         print(model.get_metrics(all_stocks, test_data, scalers, n_back, n_forward).mean())

In [ ]:
metrics = model.get_metrics(all_stocks, test_data, scalers, n_back, n_forward)

In [ ]:
metrics.mean()

In [8]:
model.save_model("./model_checkpoints/gru_100_2000_7000_07.pt")  